In [217]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [218]:
import numpy as np
from sweep.experiment import Experiment, ExperimentType
from model.data import DataModel, KFeaturesDefinition, k_features_factory
from erm.problems.problems import ProblemType
import subprocess
import logging
logging.basicConfig()

Define a datamodel

In [219]:
d = 100
vanilla_gaussian_diagonal = [(d, 1)]
x_diagonal = KFeaturesDefinition(diagonal=vanilla_gaussian_diagonal)
θ_diagonal = KFeaturesDefinition(diagonal=vanilla_gaussian_diagonal)
ω_diagonal = KFeaturesDefinition(diagonal=vanilla_gaussian_diagonal)
δ_diagonal = KFeaturesDefinition(diagonal=vanilla_gaussian_diagonal)
ν_diagonal = KFeaturesDefinition(diagonal=vanilla_gaussian_diagonal)

k_features_kwargs = {
    "x_diagonal": x_diagonal,
    "θ_diagonal": θ_diagonal,
    "ω_diagonal": ω_diagonal,
    "δ_diagonal": δ_diagonal,
    "ν_diagonal": ν_diagonal,
}

data_model = DataModel(
    d,
    normalize_matrices=False,
    data_model_factory=k_features_factory,
    factory_kwargs=k_features_kwargs,
)

Define an experiment

In [220]:
data_model

DataModel(d=100, normalize_matrices=False, data_model_factory=k_features_factory, factory_kwargs={'x_diagonal': KFeaturesDefinition(diagonal=[(100, 1)]), 'θ_diagonal': KFeaturesDefinition(diagonal=[(100, 1)]), 'ω_diagonal': KFeaturesDefinition(diagonal=[(100, 1)]), 'δ_diagonal': KFeaturesDefinition(diagonal=[(100, 1)]), 'ν_diagonal': KFeaturesDefinition(diagonal=[(100, 1)])}, name='', description='', gamma=1)

In [221]:
from sweep.experiment import NumpyEncoder, NumpyDecoder
import json

data_model_json = json.dumps(data_model, cls=NumpyEncoder)

In [222]:
data_model_json

'{"d": 100, "normalize_matrices": false, "data_model_factory_name": "k_features_factory", "factory_kwargs": {"x_diagonal": {"diagonal": [[100, 1]]}, "\\u03b8_diagonal": {"diagonal": [[100, 1]]}, "\\u03c9_diagonal": {"diagonal": [[100, 1]]}, "\\u03b4_diagonal": {"diagonal": [[100, 1]]}, "\\u03bd_diagonal": {"diagonal": [[100, 1]]}}, "name": "", "description": "", "gamma": 1}'

In [223]:
dm = DataModel.from_dict(json.loads(data_model_json, cls=NumpyDecoder))

In [224]:
dm

DataModel(d=100, normalize_matrices=False, data_model_factory=k_features_factory, factory_kwargs={'x_diagonal': KFeaturesDefinition(diagonal=[[100, 1]]), 'θ_diagonal': KFeaturesDefinition(diagonal=[[100, 1]]), 'ω_diagonal': KFeaturesDefinition(diagonal=[[100, 1]]), 'δ_diagonal': KFeaturesDefinition(diagonal=[[100, 1]]), 'ν_diagonal': KFeaturesDefinition(diagonal=[[100, 1]])}, name='', description='', gamma=1)

In [225]:
# dm.generate_model_matrices()

In [226]:
experiment = Experiment(
    state_evolution_repetitions=1,
    erm_repetitions=1,
    alphas=np.linspace(0,10,5),
    epsilons=np.array([0, 0.1]),
    lambdas=np.array([0.1]),
    taus=np.array([0.1]),
    d=d,
    experiment_type=ExperimentType.Sweep,
    data_model = data_model,
    test_against_epsilons=np.array([0,0.1]),
    erm_problem_type=ProblemType.Logistic,
    gamma_fair_error=0.01,
    name="vanilla-gaussian"
)
experiment_json = experiment.to_json()

In [227]:
with open("experiment.json", "w") as f:
    f.write(experiment_json)

In [228]:
e = Experiment.fromdict(json.loads(experiment_json, cls=NumpyDecoder))

In [229]:
e.data_model

DataModel(d=100, normalize_matrices=False, data_model_factory=k_features_factory, factory_kwargs={'x_diagonal': KFeaturesDefinition(diagonal=[[100, 1]]), 'θ_diagonal': KFeaturesDefinition(diagonal=[[100, 1]]), 'ω_diagonal': KFeaturesDefinition(diagonal=[[100, 1]]), 'δ_diagonal': KFeaturesDefinition(diagonal=[[100, 1]]), 'ν_diagonal': KFeaturesDefinition(diagonal=[[100, 1]])}, name='', description='', gamma=1)

In [230]:
command = ["pwd"]
subprocess.run(command)

/Users/kasimirtanner/Documents/LocalCode/Code_PDM/AISTATS


CompletedProcess(args=['pwd'], returncode=0)

In [231]:
venv_python = ".venv/bin/python"
command = ["mpiexec", "-n", "5", venv_python, "sweep/run_sweep.py", "--json", experiment_json, "--log-level", "DEBUG"]
subprocess.run(command)

INFO:root:The MPI comm size is 5
INFO:root:This process has rank 1
INFO:root:The MPI comm size is 5
INFO:root:This process has rank 4
INFO:root:The MPI comm size is 5
INFO:root:This process has rank 0
INFO:root:Starting Master for experiment vanilla-gaussian
INFO:root:Starting Experiment vanilla-gaussian
INFO:root:The MPI comm size is 5
INFO:root:The MPI comm size is 5
INFO:root:This process has rank 2
INFO:root:This process has rank 3
  0%|          | 0/20 [00:00<?, ?it/s]INFO:root:Starting all processes
INFO:root:Sending task 0 to 1
INFO:root:Sending task 1 to 2
INFO:root:Sending task 2 to 3
INFO:root:Sending task 3 to 4
INFO:root:All processes started - receiving results and sending new tasks
INFO:root:Starting task 2
INFO:root:Received the 0th task
DEBUG:root:DataModel(d=100, normalize_matrices=False, data_model_factory=k_features_factory, factory_kwargs={'x_diagonal': KFeaturesDefinition(diagonal=[[100, 1]]), 'θ_diagonal': KFeaturesDefinition(diagonal=[[100, 1]]), 'ω_diagonal': KF

2025-02-13 22:55:21,182 - Rank 0 - INFO - The MPI comm size is 5
2025-02-13 22:55:21,182 - Rank 0 - INFO - This process has rank 0
2025-02-13 22:55:21,182 - Rank 0 - INFO - Starting Master for experiment vanilla-gaussian
2025-02-13 22:55:21,182 - Rank 0 - INFO - Starting Experiment vanilla-gaussian
2025-02-13 22:55:21,190 - Rank 0 - INFO - Starting all processes
2025-02-13 22:55:21,190 - Rank 0 - INFO - Sending task 0 to 1
2025-02-13 22:55:21,190 - Rank 0 - INFO - Sending task 1 to 2
2025-02-13 22:55:21,190 - Rank 0 - INFO - Sending task 2 to 3
2025-02-13 22:55:21,190 - Rank 0 - INFO - Sending task 3 to 4
2025-02-13 22:55:21,190 - Rank 0 - INFO - All processes started - receiving results and sending new tasks
2025-02-13 22:55:21,190 - Rank 0 - INFO - Received the 0th task


ERROR:root:Singular matrix
Traceback (most recent call last):
  File "/Users/kasimirtanner/Documents/LocalCode/Code_PDM/AISTATS/sweep/helpers.py", line 65, in process_task
    data_model.generate_model_matrices()
  File "/Users/kasimirtanner/Documents/LocalCode/Code_PDM/AISTATS/model/data.py", line 113, in generate_model_matrices
    assumption_1 = self.Σ_x - self.Σ_x.T @ np.linalg.inv(self.Σ_x) @ self.Σ_x
                                           ^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/kasimirtanner/Documents/LocalCode/Code_PDM/AISTATS/.venv/lib/python3.11/site-packages/numpy/linalg/linalg.py", line 561, in inv
    ainv = _umath_linalg.inv(a, signature=signature, extobj=extobj)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/kasimirtanner/Documents/LocalCode/Code_PDM/AISTATS/.venv/lib/python3.11/site-packages/numpy/linalg/linalg.py", line 112, in _raise_linalgerror_singular
    raise LinAlgError("Singular matrix")
numpy.linalg.LinAlgError: Singular m

2025-02-13 22:55:22,018 - Rank 0 - INFO - Received task 1 from 1
2025-02-13 22:55:22,018 - Rank 0 - DEBUG - Singular matrix
2025-02-13 22:55:22,018 - Rank 0 - ERROR - Error Task(id=1, task_type=<TaskType.SE: 1>, erm_problem_type=None, alpha=0.0, epsilon=0.0, test_against_epsilons=[0.0, 0.1], lam=0.1, tau=0.1, d=100, data_model=DataModel(d=100, normalize_matrices=False, data_model_factory=k_features_factory, factory_kwargs={'x_diagonal': KFeaturesDefinition(diagonal=[[100, 1]]), 'θ_diagonal': KFeaturesDefinition(diagonal=[[100, 1]]), 'ω_diagonal': KFeaturesDefinition(diagonal=[[100, 1]]), 'δ_diagonal': KFeaturesDefinition(diagonal=[[100, 1]]), 'ν_diagonal': KFeaturesDefinition(diagonal=[[100, 1]])}, name='', description='', gamma=1), values={}, gamma_fair_error=0.01, gamma=1, result=LinAlgError('Singular matrix'))
2025-02-13 22:55:22,018 - Rank 0 - ERROR - Singular matrix
2025-02-13 22:55:22,019 - Rank 0 - INFO - Received the 1th task
2025-02-13 22:55:22,130 - Rank 0 - INFO - Received t

ERROR:root:Singular matrix
Traceback (most recent call last):
  File "/Users/kasimirtanner/Documents/LocalCode/Code_PDM/AISTATS/sweep/helpers.py", line 65, in process_task
    data_model.generate_model_matrices()
  File "/Users/kasimirtanner/Documents/LocalCode/Code_PDM/AISTATS/model/data.py", line 113, in generate_model_matrices
    assumption_1 = self.Σ_x - self.Σ_x.T @ np.linalg.inv(self.Σ_x) @ self.Σ_x
                                           ^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/kasimirtanner/Documents/LocalCode/Code_PDM/AISTATS/.venv/lib/python3.11/site-packages/numpy/linalg/linalg.py", line 561, in inv
    ainv = _umath_linalg.inv(a, signature=signature, extobj=extobj)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/kasimirtanner/Documents/LocalCode/Code_PDM/AISTATS/.venv/lib/python3.11/site-packages/numpy/linalg/linalg.py", line 112, in _raise_linalgerror_singular
    raise LinAlgError("Singular matrix")
numpy.linalg.LinAlgError: Singular m

...

ERROR:root:Singular matrix
Traceback (most recent call last):
  File "/Users/kasimirtanner/Documents/LocalCode/Code_PDM/AISTATS/sweep/helpers.py", line 65, in process_task
    data_model.generate_model_matrices()
  File "/Users/kasimirtanner/Documents/LocalCode/Code_PDM/AISTATS/model/data.py", line 113, in generate_model_matrices
    assumption_1 = self.Σ_x - self.Σ_x.T @ np.linalg.inv(self.Σ_x) @ self.Σ_x
                                           ^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/kasimirtanner/Documents/LocalCode/Code_PDM/AISTATS/.venv/lib/python3.11/site-packages/numpy/linalg/linalg.py", line 561, in inv
    ainv = _umath_linalg.inv(a, signature=signature, extobj=extobj)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/kasimirtanner/Documents/LocalCode/Code_PDM/AISTATS/.venv/lib/python3.11/site-packages/numpy/linalg/linalg.py", line 112, in _raise_linalgerror_singular
    raise LinAlgError("Singular matrix")
numpy.linalg.LinAlgError: Singular m

2025-02-13 22:55:25,211 - Rank 0 - INFO - Received task 17 from 1
2025-02-13 22:55:25,211 - Rank 0 - DEBUG - Singular matrix
2025-02-13 22:55:25,211 - Rank 0 - ERROR - Error Task(id=17, task_type=<TaskType.ERM: 0>, erm_problem_type=<ProblemType.Logistic: 0>, alpha=7.5, epsilon=0.0, test_against_epsilons=[0.0, 0.1], lam=0.1, tau=0.1, d=100, data_model=DataModel(d=100, normalize_matrices=False, data_model_factory=k_features_factory, factory_kwargs={'x_diagonal': KFeaturesDefinition(diagonal=[[100, 1]]), 'θ_diagonal': KFeaturesDefinition(diagonal=[[100, 1]]), 'ω_diagonal': KFeaturesDefinition(diagonal=[[100, 1]]), 'δ_diagonal': KFeaturesDefinition(diagonal=[[100, 1]]), 'ν_diagonal': KFeaturesDefinition(diagonal=[[100, 1]])}, name='', description='', gamma=1), values={}, gamma_fair_error=0.01, gamma=1, result=LinAlgError('Singular matrix'))
2025-02-13 22:55:25,211 - Rank 0 - ERROR - Singular matrix
2025-02-13 22:55:25,211 - Rank 0 - INFO - Received the 17th task
2025-02-13 22:55:25,314 - R

ERROR:root:Singular matrix
Traceback (most recent call last):
  File "/Users/kasimirtanner/Documents/LocalCode/Code_PDM/AISTATS/sweep/helpers.py", line 65, in process_task
    data_model.generate_model_matrices()
  File "/Users/kasimirtanner/Documents/LocalCode/Code_PDM/AISTATS/model/data.py", line 113, in generate_model_matrices
    assumption_1 = self.Σ_x - self.Σ_x.T @ np.linalg.inv(self.Σ_x) @ self.Σ_x
                                           ^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/kasimirtanner/Documents/LocalCode/Code_PDM/AISTATS/.venv/lib/python3.11/site-packages/numpy/linalg/linalg.py", line 561, in inv
    ainv = _umath_linalg.inv(a, signature=signature, extobj=extobj)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/kasimirtanner/Documents/LocalCode/Code_PDM/AISTATS/.venv/lib/python3.11/site-packages/numpy/linalg/linalg.py", line 112, in _raise_linalgerror_singular
    raise LinAlgError("Singular matrix")
numpy.linalg.LinAlgError: Singular m

2025-02-13 22:55:25,549 - Rank 0 - INFO - Received task 19 from 4
2025-02-13 22:55:25,549 - Rank 0 - DEBUG - Singular matrix
2025-02-13 22:55:25,549 - Rank 0 - ERROR - Error Task(id=19, task_type=<TaskType.ERM: 0>, erm_problem_type=<ProblemType.Logistic: 0>, alpha=10.0, epsilon=0.0, test_against_epsilons=[0.0, 0.1], lam=0.1, tau=0.1, d=100, data_model=DataModel(d=100, normalize_matrices=False, data_model_factory=k_features_factory, factory_kwargs={'x_diagonal': KFeaturesDefinition(diagonal=[[100, 1]]), 'θ_diagonal': KFeaturesDefinition(diagonal=[[100, 1]]), 'ω_diagonal': KFeaturesDefinition(diagonal=[[100, 1]]), 'δ_diagonal': KFeaturesDefinition(diagonal=[[100, 1]]), 'ν_diagonal': KFeaturesDefinition(diagonal=[[100, 1]])}, name='', description='', gamma=1), values={}, gamma_fair_error=0.01, gamma=1, result=LinAlgError('Singular matrix'))
2025-02-13 22:55:25,549 - Rank 0 - ERROR - Singular matrix
2025-02-13 22:55:25,549 - Rank 0 - INFO - Received the 19th task
2025-02-13 22:55:25,582 - 

CompletedProcess(args=['mpiexec', '-n', '5', '.venv/bin/python', 'sweep/run_sweep.py', '--json', '{"name": "vanilla-gaussian", "state_evolution_repetitions": 1, "erm_repetitions": 1, "alphas": [0.0, 2.5, 5.0, 7.5, 10.0], "epsilons": [0.0, 0.1], "test_against_epsilons": [0.0, 0.1], "lambdas": [0.1], "taus": [0.1], "d": 100, "experiment_type": "Sweep", "data_model": {"d": 100, "normalize_matrices": false, "data_model_factory_name": "k_features_factory", "factory_kwargs": {"x_diagonal": {"diagonal": [[100, 1]]}, "\\u03b8_diagonal": {"diagonal": [[100, 1]]}, "\\u03c9_diagonal": {"diagonal": [[100, 1]]}, "\\u03b4_diagonal": {"diagonal": [[100, 1]]}, "\\u03bd_diagonal": {"diagonal": [[100, 1]]}}, "name": "", "description": "", "gamma": 1}, "erm_problem_type": "Logistic", "gamma_fair_error": 0.01}', '--log-level', 'DEBUG'], returncode=0)